In [1]:
using DrWatson
@quickactivate

using ExponentialAction
using Roots

# Want to have different names.
# Don't like this reuse of the function name Gmatrix.

include(srcdir("KBZR-CR.jl"))
#G_KBZR_CR(v) = Gmatrix(v)

#include(srcdir("KBZR-OGD.jl"))
#function G_KBZR_OGD(v) = Gmatrix(v)

#include(srcdir("KBZR-WLMSR.jl"))
#function G_KBZR_WLMSR(v) = Gmatrix(v)

#include(srcdir("KBZR-MGWMN.jl"))
#G_KBZR_MGWMN(v) = Gmatrix(v)

param

# Hunting for Maximum Current as Easily as Possible
Suppose we have a continuous-time, discrete-state Markov chain model of a voltage-gated ion channel. Let $v$ be the transmembrane voltage and $\mathbf{G}_v$ an irreducible voltage-dependent transition rate matrix (the irreducibility gives us a unique stationary distribution). Let us fix voltage for all times $t > 0$ and suppose an initial state distribution $\left\langle \mu_0 \right|$. The expected current at time $t$ goes like

$\qquad \displaystyle
    \left\langle I(t) \right \rangle \propto \left \langle \mu(t) \mid \delta_\mathrm{O} \right \rangle
$

$\qquad \displaystyle \phantom{\left\langle I(t) \right \rangle} =
    \left\langle{\mu_0} \right| \mathrm{e}^{t\mathbf{G}_{v}} \left| \delta_\mathrm{O} \right\rangle
$,

where $\delta_\mathrm{O}$ is the index corresponding to the "open" state and $\left| \delta_\mathrm{O} \right\rangle$ is $1$ at that index and zero elsewhere. 

We wish to find $t>0$ that (locally) maximizes $\left\langle I(t) \right \rangle$. Naturally we want to take a time derivative and set it to $0$:

$\qquad \displaystyle
\frac{\mathrm{d}}{\mathrm{d}t} \left\langle I(t) \right \rangle \phantom{=}\hspace{-0.8em}\propto
    \left\langle{\mu_0} \right| \frac{\mathrm{d}}{\mathrm{d}t}\mathrm{e}^{t\mathbf{G}_{v}} \left| \delta_\mathrm{O} \right\rangle
$

$\qquad \displaystyle
\phantom{\frac{\mathrm{d}}{\mathrm{d}t} \left\langle I(t) \right \rangle \propto}\hspace{-0.8em} =
    \left\langle{\mu_0} \right| \mathrm{e}^{t\mathbf{G}_{v}} \mathbf{G}_v \left| \delta_\mathrm{O} \right\rangle
$

$\qquad \displaystyle
\phantom{\frac{\mathrm{d}}{\mathrm{d}t} \left\langle I(t) \right \rangle \propto}\hspace{-0.8em} =
    {\large\langle} \mu_0 {\large|} \mathrm{e}^{t\mathbf{G}_{v}} {\large|} {\mathbf{G}_v}_{[:,\delta_\mathrm{O}]}  {\large \rangle}
$,

where $i$ is in the state index. The first equality holds by definition of the matrix exponential (as the solution to a matrix ODE) and commutativity of a matrix with its scaled exponential; the second exploits the associativity of matrix multiplication and the fact that $\mathbf{G}_v \left| \delta_\text{O}\right\rangle$ is simply the column vector whose elements are entries of $\mathbf{G}_v$'s $\delta_\text{O}$-th column.


Now I want to write a function for finding the zeros...

In [ ]:
"""
    max_current_time(G, u0, I_index) -> t

Find the time `t` at which evolution according to a rate matrix `G` from initial state distribution `u0` maximizes `u(t)[I_index]`. 
"""
function max_current_time(G, u0, I_index)

end

5×5 Matrix{Float64}:
 -0.0336879    0.0336879   0.0           0.0         0.0
  0.00211198  -2.17411     2.172         0.0         0.0
  0.0          1.077      -1.098         0.0105012   0.0105012
  0.0          0.0         0.00275166   -0.660608    0.657856
  0.0          0.0         0.000950429   0.227225   -0.228175

## Directions
* Numerics: create a Julia function that takes in
    1. A "time" (difference) $x$,
    1. The rate matrix $\mathbf{G}$,
    1. The index $i$ of an arbitrary row/column 
    1. The index $\delta_\mathrm{O}$ of the row/column corresponding to the open state
    
    and returns $[\mathrm{e}^{x\mathbf{G}}]_{i\delta_\mathrm{O}}$. 
    
    If $\left \langle \mu_0 \right|$ has strictly positive elements (which we can set up, and which will certainly be true of any "real" $\left \langle \mu(t) \right|$ for irreducible $\mathbf{G}$, which we expect most of the time and require for a unique stationary distribution) then $\left\langle I(t) \right \rangle = 0$ only if $\frac{\mathrm{d}}{\mathrm{d}t}[\mathrm{e}^{x\mathbf{G}}]_{i\delta_\mathrm{O}} = 0$ **for each $i$ in the state index**.

    So, via the Julia function specified above, we can try to just numerically probe maxima/minima of one $i$ at a time, knowing that the local minimum we're after must correspond to a local max, min, or flat region for each $i$ simultaneously. We can use that to narrow down the right "regime" of times to look for and so save simulation time when hunting for these current maxima numerically.

* Analytics: see if we can write down a closed-form expression for the matrix exponential in terms of its spectra a-la Paul's ["Beyond the Spectral Theorem"](https://www.doi.org/10.1063/1.5040705). My first brute-force idea is to take the expression of powers of a matrix and substitute it into the power series for the matrix exponential. But my suspicion is that if Paul didn't already do this, it's either a dead end or a paper in itself. I (Mikhael) will ping him.